In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
# df_circuits=spark.read.parquet(f"{processed_folder_path}/circuits").withColumnRenamed("location","circuit_location")
# df_drivers=spark.read.parquet(f"{processed_folder_path}/drivers").withColumnRenamed("number","driver_number").withColumnRenamed("name","driver_name").withColumnRenamed("nationality","driver_nationality")
# df_constructors=spark.read.parquet(f"{processed_folder_path}/constructors").withColumnRenamed("name","team")
# df_results=spark.read.parquet(f"{processed_folder_path}/results").filter(f"file_date='{v_file_date}'").withColumnRenamed('time','race_time').withColumnRenamed("race_id","result_race_id")
# df_races=spark.read.parquet(f"{processed_folder_path}/races").withColumnRenamed("name","race_name").withColumnRenamed('race_timestamp','race_date')

In [0]:
# To fetch Data in processed layer in delta form
df_circuits=spark.read.format("delta").load(f"{processed_folder_path}/circuits").withColumnRenamed("location","circuit_location")
df_drivers=spark.read.format("delta").load(f"{processed_folder_path}/drivers").withColumnRenamed("number","driver_number").withColumnRenamed("name","driver_name").withColumnRenamed("nationality","driver_nationality")
df_constructors=spark.read.format("delta").load(f"{processed_folder_path}/constructors").withColumnRenamed("name","team")
df_results=spark.read.format("delta").load(f"{processed_folder_path}/results").filter(f"file_date='{v_file_date}'").withColumnRenamed('time','race_time').withColumnRenamed("race_id","result_race_id")
df_races=spark.read.format("delta").load(f"{processed_folder_path}/races").withColumnRenamed("name","race_name").withColumnRenamed('race_timestamp','race_date')

In [0]:
display(df_constructors)

In [0]:
display(df_results)

In [0]:
race_ciricuits_df=df_races.join(df_circuits,df_races.circuit_id==df_circuits.circuit_id,how='inner').select(df_races.race_id,df_races.race_name,df_races.race_date,df_races.race_year,df_circuits.circuit_location)

In [0]:
from pyspark.sql.functions import current_timestamp
race_results_df=df_results.join(race_ciricuits_df,race_ciricuits_df.race_id==df_results.result_race_id,how='inner').join(df_drivers,df_results.driver_id==df_drivers.driver_id,how='inner').join(df_constructors,df_results.constructor_id==df_constructors.constructor_Id,how='inner').withColumn("created_date",current_timestamp()).select("race_year","race_name","race_date","circuit_location","driver_name","driver_number","driver_nationality","team","grid","fastest_lap","race_time","points",'position',"created_date",'race_id')

In [0]:
display(race_results_df)

In [0]:
#race_results_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/race_results")

In [0]:
#Method 2 of Incremental 
# spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [0]:
#Method 2 of Incremental
# if(spark._jsparkSession.catalog().tableExists('f1_presentation.race_results')):
#     race_results_df.write.mode('overwrite').insertInto('f1_presentation.race_results')
# else:
#     race_results_df.write.mode("overwrite").partitionBy('race_id').format('parquet').saveAsTable('f1_presentation.race_results')    

In [0]:
#Save as Table in SQL
#race_results_df.write.mode('overwrite').format('parquet').saveAsTable('f1_presentation.race_results')

In [0]:
#for delta
merge_condition="tgt.race_id=src.race_id AND tgt.driver_name=src.driver_name"
merge_delta_data(race_results_df,'f1_presentation','race_results',presentation_folder_path,merge_condition,'race_id')

In [0]:
%sql
Select race_id,count(*) 
from f1_presentation.race_results
group by race_id
order by race_id 

In [0]:
dbutils.notebook.exit("success")